In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime

# Data Ingestion 
Data is ingested and the index for the dataset is set to `Date` column

In [ ]:
daily_cases = pd.read_csv('../../cleaned_datasets/india/daily_cases_india.csv')
daily_cases['Date'] = pd.to_datetime(daily_cases['Date'], format = '%Y-%m-%d')
daily_cases = daily_cases[:-1]
daily_cases

In [ ]:
indexed = daily_cases.set_index('Date')
indexed

In [ ]:
indexed['Confirmed'].plot()

In [ ]:
indexed['Deaths'].plot()

In [ ]:
indexed['Recovered'].plot()

In [ ]:
indexed['Active'].plot()

**Train Test Split**     
Keep upto `08-08-2021` or 0.895 of overall timeseries as train for timeseries and beyond that for forecasting. This also eliminates the outlier caused due to missing data in `Recovered`

In [ ]:
# train = indexed[:"2021-08-08"]
# val = indexed["2021-08-09":]

splitlen = int(0.9*len(daily_cases))

# train = daily_cases[:splitlen-3]
# val = daily_cases[splitlen+1-3:]

train = daily_cases[:splitlen]
val = daily_cases[splitlen:]

train = train.set_index('Date')
val = val.set_index('Date')

# train = indexed[:"2021-07-04"]
# val = indexed["2021-07-05":]

In [ ]:
val.plot()

In [ ]:
train['Recovered'].plot()

In [ ]:
confirmed_ts = train['Confirmed'].dropna()

In [ ]:
#Unvariate Time Series for Confirmed Cases 
def roll_stats(ts, window):
  ''' Function to find rolling mean and rolling std dev and plot them'''
  rollmean = ts.rolling(window = window).mean()
  rollstd = ts.rolling(window = window).std()
  print(rollmean, rollstd)

  close = plt.plot(ts, color = 'blue', label = 'Original')
  mean = plt.plot(rollmean, color = 'red', label = 'Rolling Mean')
  std = plt.plot(rollstd, color = 'green', label = 'Rolling Standard Dev')
  plt.legend(loc = 'best')
  plt.title('Rolling Statistics for Confirmed')
  plt.show()

In [ ]:
roll_stats(confirmed_ts, 30)

In [ ]:
roll_stats(confirmed_ts, 90)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def run_dicky_fuller(ts):
  '''Function to run Augmented Dicky Fuller test on the passed time series and report the statistics from the test'''
  print("Observations of Dickey-fuller test")
  dftest = adfuller(ts,autolag='AIC')
  dfoutput=pd.Series(dftest[0:4],index=['Test Statistic','p-value','#lags used','number of observations used'])

  for key,value in dftest[4].items():
      dfoutput['critical value (%s)'%key]= value
  print(dfoutput)

In [ ]:
run_dicky_fuller(confirmed_ts)

The original time-series is **Non-stationary**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(confirmed_ts, model='additive', freq=10)
fig = plt.figure()  
fig = decomp.plot()  
fig.set_size_inches(16, 9)

In [ ]:
diff = confirmed_ts.diff() 
roll_stats(diff, 30)

In [ ]:
diff = confirmed_ts.diff() 
roll_stats(diff, 90)

In [ ]:
run_dicky_fuller(diff.dropna())

We see that after differencing the time-series looks stationary both in the rolling statistics plot as well as the Dicky-Fuller test

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(diff.dropna(), lags=50, ax = ax1)
ax2 = fig.add_subplot(212)
fig = plot_pacf(diff.dropna(), lags=50, ax = ax2)

(p,q) = (1,1), (1,2), (2,1), (2,2) seem viable.

## ARIMA

In [ ]:
from statsmodels.tsa.arima_model import ARIMA 

# ARIMA (3,1,2) and (1,1,2) show similar results but (3,1,2) chosen due to lower AIC value

# ARIMA(p,d,q) = (3,1,2)
model_ARIMA = ARIMA(confirmed_ts, order=(3,1,2))
results_ARIMA = model_ARIMA.fit()
results_ARIMA.summary()

In [ ]:
results_ARIMA.plot_predict(start = 100, end = 900, dynamic = False);

In [ ]:
# ARIMA(p,d,q) = (1,1,0) - AR model
model_AR = ARIMA(confirmed_ts, order=(1,1,0))
results_AR = model_AR.fit()
results_AR.summary()

In [ ]:
results_AR.plot_predict(start = 100, end = 900, dynamic = False);

In [ ]:
# ARIMA(p,d,q) = (0,1,1) - MA model
model_MA = ARIMA(confirmed_ts, order=(0,1,1))
results_MA = model_MA.fit()
results_MA.summary()

In [ ]:
results_MA.plot_predict(start = 100, end = 900, dynamic = False);

Comparing AIC values of ARIMA, AR and MA we can see that ARIMA is the best, so we proceed with ARIMA

In [ ]:
fc, se, conf = results_ARIMA.forecast(len(val), alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=val.index)
lower_series = pd.Series(conf[:, 0], index=val.index)
upper_series = pd.Series(conf[:, 1], index=val.index)

# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(confirmed_ts, label='training')
plt.plot(val['Confirmed'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
# plt.plot(lower_series, linestyle = '--', color = 'grey', label = '95% Confidence Interval')
# plt.plot(upper_series, linestyle = '--', color = 'grey')
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()
# plt.savefig('../../figures/india_arima.eps', format='eps')

In [ ]:
# Plot
plt.figure(figsize=(8,5), dpi=100)
plt.plot(val['Confirmed'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

## Auto-ARIMA

In [ ]:
#!pip install pmdarima

In [ ]:
import pmdarima 

In [ ]:
model_autoARIMA = pmdarima.auto_arima(confirmed_ts)
model_autoARIMA.get_params()

We see that auto-ARIMA picked out (p,d,q) = (3,1,2) for this particular train-val split, but for others it was noticed that (1,1,2) was picked. As the ACF and PACF also suggest (1,1,2) we go with that.

In [ ]:
fc = model_autoARIMA.predict(n_periods=len(val))

# Make as pandas series
fc_series = pd.Series(fc, index=val.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(confirmed_ts, label='training')
plt.plot(val['Confirmed'], label='actual')
plt.plot(fc_series, label='forecast')

plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

For ARIMA we see that ARIMA(3,1,2) or ARIMA(1,1,2) is the most optimal, but still both of them are not accurate in predicting spikes

## SARIMAX

We now try SARIMAX, with p,d,q = (1,1,2). But getting the seasonal order (P,D,Q,S) is not obvious from ACF and PACF. So we will apply GridSearch to find the most optimal SARIMAX(p,d,q)(P,D,Q,S) model

In [ ]:
model_autoSARIMA = pmdarima.auto_arima(confirmed_ts, seasonal=True)
model_autoSARIMA.get_params()

Even with seasonal ARIMA considered, we see that seasonal models aren't as effective, as `auto_arima` has predicted seasonal_order of (0,0,0,0). The models are tested based on AIC internally and SARIMA with seasonal components seem to have performed worse than non-seasonal ARIMA

In [ ]:
fc = model_autoSARIMA.predict(n_periods=len(val))

# Make as pandas series
fc_series = pd.Series(fc, index=val.index)
# lower_series = pd.Series(conf[:, 0], index=val.index)
# upper_series = pd.Series(conf[:, 1], index=val.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(confirmed_ts, label='training')
plt.plot(val['Confirmed'], label='actual')
plt.plot(fc_series, label='forecast')
# plt.fill_between(lower_series.index, lower_series, upper_series, 
#                  color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
## DO NOT RUN THIS CELL

# import statsmodels.api as sm
# import itertools
# # from sm.tsa.statespace import SARIMAX

# def sarimax(ts,pdq,seasonal_pdq):
#     results = []
#     for order in pdq:
#         for seas in seasonal_pdq:
#             print(order, seas)
#             try:
#                 mod = sm.tsa.statespace.SARIMAX(ts,
#                               order=order,
#                               seasonal_order=seas)
#                 res = mod.fit()
#                 results.append((res,res.aic,param))
#                 print('Tried out SARIMAX{}x{} - AIC:{}'.format(param[0], param[1], round(res.aic,2)))
#             except Exception as e:
#                 print(e)
            
#     return results
# # set parameter range
# # p,d,q = range(0,3),[1],range(0,3)
# P,D,Q,s = range(0,2),[0],range(0,2),[250]
# # list of all parameter combos
# pdq = [(1,1,2)]
# seasonal_pdq = list(itertools.product(P, D, Q, s))
# # all_param = list(itertools.product(pdq,seasonal_pdq))
# # all_param = [(pdq, s) for s in seasonal_pdq]
# # for param in all_param:
# #     print(param)

# all_res = sarimax(confirmed_ts,pdq, seasonal_pdq)

# ARIMA + GARCH

In [ ]:
# Get the residuals from the ARIMA(1,1,2) model fit earlier 
resid = results_ARIMA.resid 
resid.plot()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(resid.dropna(), lags=60, ax = ax1)
ax2 = fig.add_subplot(212)
fig = plot_pacf(resid.dropna(), lags=60, ax = ax2)

In [ ]:
from arch import arch_model

resid_GARCH = arch_model(resid, p=1, q=1)
garch_fit = resid_GARCH.fit()

In [ ]:
garch_fit.summary()

In [ ]:
resid_forecasts = garch_fit.forecast(horizon=len(val))
resid_fc = resid_forecasts.residual_variance.values[-1, :]
resid_fc
resid_fc = np.sqrt(resid_fc)

In [ ]:
fc, se, conf = results_ARIMA.forecast(len(val), alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=val.index)
lower_series = pd.Series(conf[:, 0], index=val.index)
upper_series = pd.Series(conf[:, 1], index=val.index)

# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(confirmed_ts, label='training')
plt.plot(val['Confirmed'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, fc_series + resid_fc, fc_series - resid_fc, 
                 color='k', alpha=.15)
# plt.plot(fc_series + resid_fc, linestyle = '--', color = 'grey', label = '95% Confidence Interval')
# plt.plot(fc_series - resid_fc, linestyle = '--', color = 'grey')
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()
# plt.savefig('../../figures/india_arima+garch.eps', format='eps')

We see that the Confidence Interval is now narrowed down and isn't exploding like earlier. GARCH has made the variance predictable hence the narrower CI. Without GARCH the CI was exponentially expanding (as can be seen earlier) which would lead to inaccurate prediction of variance

In [ ]:
# Make as pandas series
fc_series = pd.Series(fc, index=val.index)
lower_series = pd.Series(conf[:, 0], index=val.index)
upper_series = pd.Series(conf[:, 1], index=val.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(val['Confirmed'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, fc_series + resid_fc, fc_series - resid_fc, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

## Rolling forecasts (Short-term)

Here only the next days cases is predicted and the data is given to the ARIMA model as it comes in to predict the following days cases

In [ ]:
history = confirmed_ts.copy()
print(history)

In [ ]:
roll_fc = pd.Series(index = val.index)
roll_resid = pd.Series(index = val.index)

In [ ]:
#Only rolling ARIMA

for exp in val['Confirmed']:
    model = ARIMA(history, order=(3,1,2))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    newindex = history.index[-1] + pd.to_timedelta(1, 'D')
    roll_fc[newindex] = yhat[0]
    history[newindex] = exp

In [ ]:
#Rolling ARIMA + rolling GARCH

# for exp in val['Confirmed']:
#     model = ARIMA(history, order=(1,1,2))
#     model_fit = model.fit()
#     output = model_fit.forecast()
#     resid = model_fit.resid
#     roll_garch = arch_model(resid, p=1, q=1)
#     roll_garch_fit = roll_garch.fit(disp=-1)
#     garch_fc = roll_garch_fit.forecast().residual_variance.values[-1, :]
#     yhat = output[0]
#     newindex = history.index[-1] + pd.to_timedelta(1, 'D')
#     roll_fc[newindex] = yhat[0]
#     history[newindex] = exp
#     roll_resid[newindex] = np.sqrt(garch_fc)[0]

In [ ]:
roll_fc.plot()
print(val['Confirmed'])
roll_fc

In [ ]:
lower_series = pd.Series(conf[:, 0], index=val.index)
upper_series = pd.Series(conf[:, 1], index=val.index)

# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(confirmed_ts, label='training')
plt.plot(val['Confirmed'], label='actual')
plt.plot(roll_fc, label='forecast')
# plt.fill_between(lower_series.index, roll_fc + resid_fc, roll_fc - resid_fc, 
#                  color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()
# plt.savefig('../../figures/india_arima_rolling.eps', format='eps')

In [ ]:
lower_series = pd.Series(conf[:, 0], index=val.index)
upper_series = pd.Series(conf[:, 1], index=val.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(val['Confirmed'], label='actual')
plt.plot(roll_fc, label='forecast')
# plt.fill_between(lower_series.index, roll_fc + resid_fc, roll_fc - resid_fc, 
#                  color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# lower_series = pd.Series(conf[:, 0], index=val.index)
# upper_series = pd.Series(conf[:, 1], index=val.index)

# # Plot
# plt.figure(figsize=(12,5), dpi=100)
# plt.plot(val['Confirmed'], label='actual')
# plt.plot(roll_fc, label='forecast')
# plt.fill_between(lower_series.index, roll_fc + roll_resid, roll_fc - roll_resid, 
#                  color='k', alpha=.15)
# plt.title('Forecast vs Actuals')
# plt.legend(loc='upper left', fontsize=8)
# plt.show()

## Evaluation Metrics

MAPE and MAE used

Comparing both short-term and long-term forecasts

In [ ]:
def MAPE(Y_actual,Y_Predicted, title):
    mask = Y_actual != 0
    
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual)[mask])*100
#     print(mape)
    print(f"MAPE of {title} is {mape}%")
#     return mape[mape.index[0]]


mape_fc = MAPE(val['Confirmed'], fc_series, title="Long-term")
mape_roll = MAPE(val['Confirmed'], roll_fc, title="Short-term (rolling)")
# mape_cases = MAPE(test_original[['undiff_Confirmed']], fore_original[['undiff_Confirmed']], title="Daily cases")

In [ ]:
from tensorflow.keras.losses import MeanAbsolutePercentageError, MeanAbsoluteError

mape_keras = MeanAbsolutePercentageError() 

print(mape_keras(val['Confirmed'], fc_series).numpy())
print(mape_keras(val['Confirmed'], roll_fc).numpy())

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE of Long-term:', mean_absolute_error(val['Confirmed'], fc_series))
print('MAE of short-term:', mean_absolute_error(val['Confirmed'], roll_fc))